In [1]:
import jax
import jax.numpy as jnp

import numpy as np

jax.config.update("jax_enable_x64", True)

N_SAMPLES = 100

This is the notebook comparing all inner functions of `SkyFrameToDetectorFrameSkyPositionTransform` in `Jim` and the corresponding functions in `bilby`. 

The following is the test comparing `theta_phi_to_ra_dec` in `jim` and that in `bilby`.
See [bilby's lines](<https://git.ligo.org/lscsoft/bilby/-/blob/c6bcb81649b7ebf97ae6e1fd689e8712fe028eb0/bilby/core/utils/conversion.py#:~:text=def-,theta_phi_to_ra_dec,-(theta%2C>)

In [2]:
# test theta_phi_to_ra_dec in inverse transform
from jimgw.single_event.utils import theta_phi_to_ra_dec as jim_theta_phi_to_ra_dec
from bilby.core.utils import theta_phi_to_ra_dec as bilby_theta_phi_to_ra_dec

key = jax.random.PRNGKey(42)

tol_diff_ra = 0
tol_diff_dec = 0

for _ in range(N_SAMPLES):
    key, subkey = jax.random.split(key)
    subkeys = jax.random.split(subkey, 3)
    theta = jax.random.uniform(subkeys[0], (1,), minval=0, maxval=jnp.pi)
    phi = jax.random.uniform(subkeys[1], (1,), minval=0, maxval= jnp.pi)
    gmst = jax.random.uniform(subkeys[2], (1,), minval=0, maxval=2 * jnp.pi)   

    jim_ra, jim_dec = jim_theta_phi_to_ra_dec(theta, phi, gmst)
    bilby_ra, bilby_dec = bilby_theta_phi_to_ra_dec(theta, phi, gmst)
    bilby_ra = bilby_ra % (2 * jnp.pi)
    diff_ra = jnp.abs(jim_ra - bilby_ra)
    diff_dec = jnp.abs(jim_dec - bilby_dec)
    tol_diff_ra += diff_ra
    tol_diff_dec += diff_dec
    
    assert jnp.allclose(jim_ra,bilby_ra, atol=1e-5), f"jim_ra: {jim_ra}, bilby_ra: {bilby_ra}"
    assert jnp.allclose(jim_dec,bilby_dec, atol=1e-5), f"jim_dec: {jim_dec}, bilby_dec: {bilby_dec}"

mean_ra_diff = tol_diff_ra / N_SAMPLES
mean_dec_diff = tol_diff_dec / N_SAMPLES
print("mean_ra_diff", mean_ra_diff)
print("mean_dec_diff", mean_dec_diff)

mean_ra_diff [0.]
mean_dec_diff [0.]


The following is the test comparing `angle_rotation` in `jim` and `zenith_azimuth_to_theta_phi` in `bilby`. See [bilby's lines](https://git.ligo.org/colm.talbot/bilby-cython/-/blob/main/bilby_cython/geometry.pyx?ref_type=heads#:~:text=zenith_azimuth_to_theta_phi)

In [3]:
# test angle rotation
from jimgw.single_event.utils import angle_rotation as jim_angle_rotation
from jimgw.single_event.utils import euler_rotation 
from bilby_cython.geometry import zenith_azimuth_to_theta_phi as bibly_angle_rotation
from bilby_cython.geometry import rotation_matrix_from_delta 

tol_diff_theta = 0
tol_diff_phi = 0

for _ in range(N_SAMPLES):
    zenith = np.random.uniform(0, np.pi)
    azimuth = np.random.uniform(0, 2 * np.pi)
    delta_x = np.random.uniform(0, 1, size=3)
    
    # Ensure rotation matrix are the same
    jim_rot = euler_rotation(delta_x)
    bilby_rot = rotation_matrix_from_delta(delta_x)
    
    assert jnp.allclose(jim_rot, bilby_rot), f"jim_rot: {jim_rot}, bilby_rot: {bilby_rot}"

    jim_theta, jim_phi= jim_angle_rotation(zenith, azimuth, jim_rot)
    bilby_out = bibly_angle_rotation(zenith, azimuth, delta_x)
 
    diff_theta = jnp.abs(jim_theta - bilby_out[0])
    diff_phi = jnp.abs(jim_phi - bilby_out[1])
    
    tol_diff_theta += diff_theta
    tol_diff_phi += diff_phi
    
    assert jnp.allclose(jim_theta, bilby_out[0]), f"jim_theta: {jim_theta}, bilby_theta: {bilby_out[0]}"
    assert jnp.allclose(jim_phi, bilby_out[1]), f"jim_phi: {jim_phi}, bilby_phi: {bilby_out[1]}"

mean_diff_theta = tol_diff_theta / N_SAMPLES
mean_diff_phi = tol_diff_phi / N_SAMPLES
print("mean_diff_theta", mean_diff_theta)
print("mean_diff_phi", mean_diff_phi)

mean_diff_theta 6.522560269672795e-17
mean_diff_phi 8.881784197001253e-18


The following compares `delta_x` in `Jim` and that in `bilby`

In [7]:
# test delta_x
from itertools import combinations
from bilby.gw.detector import InterferometerList
from jimgw.single_event.detector import detector_preset

HLV = ['H1', 'L1', 'V1']
for ifos in combinations(HLV, 2):
    jim_ifos = [detector_preset[ifo] for ifo in ifos]
    bilby_ifos = InterferometerList(ifos)

    delta_x_j = jim_ifos[0].vertex - jim_ifos[1].vertex
    delta_x_b = bilby_ifos[0].vertex - bilby_ifos[1].vertex

    print(f"diff_delta_x of ({ifos[0]}, {ifos[1]})", delta_x_j - delta_x_b)

diff_delta_x of (H1, L1) [1.86264515e-09 4.65661287e-10 0.00000000e+00]
diff_delta_x of (H1, V1) [0.00000000e+00 9.31322575e-10 0.00000000e+00]
diff_delta_x of (L1, V1) [-9.31322575e-10  0.00000000e+00  0.00000000e+00]


The following compares the `gmst` in `Jim` and that in `bilby`. See [bilby's lines](https://git.ligo.org/colm.talbot/bilby-cython/-/blob/main/bilby_cython/time.pyx?ref_type=heads#:~:text=greenwich_mean_sidereal_time)

In [5]:
from bilby_cython.time import greenwich_mean_sidereal_time
from astropy.time import Time

tol_diff = 0
for time in np.random.uniform(1, 10000000, N_SAMPLES):
    gmst_j = Time(time, format="gps").sidereal_time("apparent", "greenwich").rad % (2 * np.pi)
    gmst_b = greenwich_mean_sidereal_time(time) % (2 * np.pi)
    diff = jnp.abs(gmst_j - gmst_b)
    tol_diff += diff
    
mean_diff = tol_diff / N_SAMPLES
print("mean_diff", mean_diff)

mean_diff 1.081922923157029e-05


The following compares the `SkyFrameToDetectorFrameSkyPositionTransform` in `Jim` and `zenith_azimuth_to_ra_dec` in `bilby`. See [bilby's lines](https://git.ligo.org/lscsoft/bilby/-/blob/c6bcb81649b7ebf97ae6e1fd689e8712fe028eb0/bilby/gw/utils.py#:~:text=zenith_azimuth_to_ra_dec)

In [6]:
# test the transform 
from jimgw.single_event.transforms import SkyFrameToDetectorFrameSkyPositionTransform
from bilby.gw.utils import zenith_azimuth_to_ra_dec as bilby_zenith_azimuth_to_ra_dec
from jimgw.single_event.detector import H1, L1
import jax
import jax.numpy as jnp
import bilby

jax.config.update("jax_enable_x64", True)

key = jax.random.PRNGKey(42)
gps_time = 1126259642.413
jim_ifos = [H1, L1]

ifo_names = ['H1', 'L1']
bilby_ifos = bilby.gw.detector.InterferometerList(ifo_names)

tol_diff_dec = 0
tol_diff_ra = 0

for _ in range(N_SAMPLES):
    key, subkey = jax.random.split(key)
    subkeys = jax.random.split(subkey, 2)
    zenith = jax.random.uniform(subkeys[0], (1,), minval=0, maxval=jnp.pi)
    azimuth = jax.random.uniform(subkeys[1], (1,), minval=0, maxval=2 * jnp.pi)
    
    jim_transform = SkyFrameToDetectorFrameSkyPositionTransform(
        gps_time=gps_time,
        ifos=jim_ifos
    )
    jim_outputs, _ = jim_transform.inverse(dict(zenith=zenith, azimuth=azimuth))
    bilby_ra, bilby_dec = bilby_zenith_azimuth_to_ra_dec(zenith[0], azimuth[0], gps_time, bilby_ifos)
    jim_ra = jim_outputs["ra"]
    jim_dec = jim_outputs["dec"]
    
    diff_ra = jnp.abs(jim_ra - bilby_ra)
    diff_dec = jnp.abs(jim_dec - bilby_dec)
    tol_diff_ra += diff_ra
    tol_diff_dec += diff_dec
    
mean_ra_diff = tol_diff_ra / N_SAMPLES
mean_dec_diff = tol_diff_dec / N_SAMPLES
print("mean_ra_diff", mean_ra_diff)
print("mean_dec_diff", mean_dec_diff)

mean_ra_diff [2.12414525e-05]
mean_dec_diff [4.88498131e-16]


As seen in the above, the source of error in `SkyFrameToDetectorFrameSkyPositionTransform` would be the difference in calculating `gmst`. `Jim` and `bilby` use different algorithms for calculating `gmst`. This introduces an error of the order 1e-5 to `ra`. 